#Method 1

In [52]:
from imutils.video import VideoStream
import imutils
import cv2
import numpy as np
import copy
from matplotlib import pyplot as plt

In [2]:
def init_data(point):
    data = {}
    data["points"] = [point]
    data["last_slope"] = None
    return data

def same_point(point1, point2):
    if point1 == point2:
        return True
    if point1[0] == point2[0] and point1[1] != point2[1]:
        return None  # corresponds to infinite slope
    else:
        return False

In [3]:
def get_slope(point1, point2):
    return (point1[1]-point2[1])/(point1[0]-point2[0])

def proximity(point1, point2, R=50):
    if (point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 <= R**2:
        return True
    else:
        return False

In [84]:
def update_line(point, data):
    """
    Returns the updated line (data) with all the points and last slope. If the point does not lie within this 
    "band" then returns the unchanged line (data)
    """
    list_of_previous_points = data["points"]
    last_slope = data["last_slope"]
    last_point = list_of_previous_points[-1]
    initial_point = list_of_previous_points[0]

    if proximity(point, last_point) is True:  # point is in close proximity
        if len(list_of_previous_points) >= 2:
            if same_point(point, last_point) is False:
                    slope_to_test = get_slope(point, last_point)
                    if same_point(point, initial_point) is False:
                        slope_init = get_slope(point, initial_point)
                        if check_slope(slope_init, last_slope, slope_to_test) is True:
                            print("passed constraints")
                            list_of_previous_points.append(point)
                            # update dict
                            data["points"] = list_of_previous_points
                            data["last_slope"] = slope_to_test

            if same_point(point, last_point) is True:
                list_of_previous_points.append(point)
                data["points"] = list_of_previous_points

        if len(list_of_previous_points) == 1:
            if same_point(point, initial_point) is False:
                slope = (point[1]-initial_point[1])/(point[0]-initial_point[0])
                # update dict
                data["last_slope"] = slope
                
            list_of_previous_points.append(point)
            data["points"] = list_of_previous_points
    
    return data
            
        
def check_slope(slope1, slope2, slope3, slope_error=2):
    """
    checks if slope3 is similar to slopes 2 and 3.   
    """
    if slope2 is None:
        if abs(slope1 - slope3) <= slope_error:
            return True
    if slope2 is not None:
        if abs(slope1 - slope3) <= slope_error and abs(slope2 - slope3) <= slope_error:
            return True
    else:
        return False

In [5]:
def check_for_lines(datas,point):
    if bool(datas) is False:  # no data points yet
        data = init_data(point)
        datas.append(data)
    else:
        count = 0  # used to keep in memory the amount of times a point was part of a line
        for i in range(len(datas)):
            line = copy.deepcopy(datas[i])
            new_line = datas[i]
            update_line(point,new_line)
            if line != new_line:
                count += 1
                
        if count == 0: # the point didn't correspond to any existing line
            data = init_data(point)
            datas.append(data)  # create a new line

    return datas

In [118]:
def to_unit_vector(point, width=1980, height=1080, AOV=np.radians(44.4)):
    angle_per_px = AOV/(width**2 + height**2)**0.5
    diagonal_length = ((point[0]-height/2)**2 + (point[1]-width/2)**2)**0.5
    angle = diagonal_length*angle_per_px
    return np.degrees(angle)

to_unit_vector([0,0])

22.2

In [85]:
video_location = 'Earth_ball.avi'
vs = cv2.VideoCapture(video_location)

In [86]:
# initialize the first frame in the video stream
avg = None
# initialise data on debris lines
datas = []

In [87]:
#main body
accumulated_thresh = np.zeros((1080,1920))
while True:
    frame = vs.read()
    frame = frame[1]
    text = "unoccupied"
    
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    # resize the frame, convert it to grayscale, and blur it
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    # if the first frame is None, initialize it
    if avg is None:
        avg = gray.copy().astype("float")
        continue

    # compute the absolute difference between the current frame and
    # first frame
    cv2.accumulateWeighted(gray, avg, 0.5)
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE,
    	cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # save the contours into an accumulated thresh image
    accumulated_thresh = cv2.accumulate(thresh, accumulated_thresh)

	# loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 200:
		# compute the bounding box for the contour, draw it on the frame,
		# and update the text
            """
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            """
            ellipse = cv2.fitEllipse(c)
            cv2.ellipse(frame,ellipse,(0,255,0),2)
            text = "Occupied"
            # get the location of the centroid
            M = cv2.moments(c)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            print("-------------------------------")
            print([cx,cy])
            datas = check_for_lines(datas, [cx,cy])
            print("-------------------------------")

        

    # draw the text and timestamp on the frame
    #cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
    #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    #cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
    #(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

	# show the frame and record if the user presses a key
    """
    cv2.imshow("Satellite Feed", frame)
    cv2.imshow("accumulated", accumulated_thresh)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    cv2.waitKey(0)
    """
    
cv2.destroyAllWindows()

-------------------------------
[994, 539]
-------------------------------
-------------------------------
[994, 539]
-------------------------------
-------------------------------
[999, 539]
passed constraints
-------------------------------
-------------------------------
[1009, 538]
passed constraints
-------------------------------
-------------------------------
[1018, 537]
passed constraints
-------------------------------
-------------------------------
[1027, 536]
passed constraints
-------------------------------
-------------------------------
[1036, 535]
passed constraints
-------------------------------
-------------------------------
[1045, 533]
passed constraints
-------------------------------
-------------------------------
[1053, 532]
passed constraints
-------------------------------
-------------------------------
[1061, 531]
passed constraints
-------------------------------
-------------------------------
[1069, 531]
passed constraints
----------------------------

In [98]:
datas_copy = copy.deepcopy(datas)
datas_new = copy.deepcopy(datas)

In [99]:
for dict in datas_copy:
    if len(dict["points"]) <= 15:
        print(len(dict["points"]))
        print("-----------------")
        print(dict)
        datas_new.remove(dict)

2
-----------------
{'points': [[776, 940], [783, 942]], 'last_slope': 0.2857142857142857}
11
-----------------
{'points': [[897, 769], [902, 772], [906, 771], [908, 770], [915, 771], [918, 769], [920, 770], [925, 770], [935, 769], [941, 768], [948, 768]], 'last_slope': 0.0}
7
-----------------
{'points': [[660, 768], [632, 732], [662, 769], [632, 730], [645, 744], [654, 754], [615, 757]], 'last_slope': -0.07692307692307693}
6
-----------------
{'points': [[700, 685], [709, 659], [718, 642], [723, 623], [728, 606], [730, 601]], 'last_slope': -2.5}
4
-----------------
{'points': [[903, 654], [906, 653], [913, 663], [887, 621]], 'last_slope': 1.6153846153846154}
15
-----------------
{'points': [[597, 606], [597, 600], [608, 619], [619, 620], [627, 618], [607, 599], [629, 616], [631, 617], [610, 597], [633, 615], [613, 598], [625, 588], [621, 595], [620, 597], [633, 597]], 'last_slope': 0.0}
7
-----------------
{'points': [[673, 434], [645, 398], [675, 434], [677, 435], [679, 435], [681, 

In [100]:
i = 0

In [110]:
print(i)
points = datas_new[i]["points"]
print(points)
plt.plot(points[0], points[1])
plt.show()
i +=1

9


IndexError: list index out of range

#Method 2

In [9]:
# KNN
backSub = cv2.createBackgroundSubtractorKNN()

In [10]:
# MOG2
backSub = cv2.createBackgroundSubtractorMOG2()

In [11]:
while True:
    ret, frame = vs.read()
    if frame is None:
        break

    fgMask = backSub.apply(frame)

    thresh = cv2.dilate(fgMask, None, iterations=2)  # makes bright regions grow
    cnts = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) > 200:
            # compute the bounding box for the contour, draw it on the frame
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            text = "Occupied"

    cv2.rectangle(frame, (10, 2), (100, 20), (255, 255, 255), 1)
   
    cv2.imshow('Frame', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('FG Mask', fgMask)

    cv2.waitKey(0)
    
cv2.destroyAllWindows()